#Imports

In [ ]:
!pip install twitter
!pip install tweepy
!pip install jsonlines
!pip install deepface
!pip install icecream

In [ ]:
import tweepy
import re
from twitter import *
import requests
import json
import csv
import jsonlines
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

#Setup

In [ ]:
# API keys
api_key = ""
api_secrets = ""
bearer_token = ""
access_token = ""
access_secret = ""
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)
 
try:
    api.verify_credentials()
    print('Successful Authentication')
except:
    print('Failed authentication')

Successful Authentication


In [ ]:
path = "/content/drive/My Drive/tweets-dataset/"

#change_GS_data.py

In [ ]:
gs_scholars_np = np.load(path + "gs_scholars_new.npy", allow_pickle=True)
# for i in range(10):
#   print(gs_scholars[i]['name'], gs_scholars[i]['url'][len("https://scholar.google.com/citations?hl=en&user="):])
print(gs_scholars_np[0].keys())
meta_info = [[],[],[]] # name, url, urls
for i in gs_scholars_np:
    meta_info[0].append(i['name'])
    meta_info[1].append(i['url'].split("user=")[-1])
    text = ""
    urls = []
    meta_info[2].append(urls)

print(meta_info[0][0:10])
print(meta_info[1][0:10])
print(meta_info[2][0:10])
a = np.array(meta_info)
np.save(path + "meta_info.npy", meta_info)

# Eval data

In [ ]:
def get_gs_id(url):
  if url != url:
    return url
  else:
    return url[len("https://scholar.google.com/citations?hl=en&user="):]
    
def get_t_id(url):
  if url != url:
    return url
  else:
    return url[len("https://twitter.com/"):]
    
gsid_2_tid_eval = pd.read_csv(path + "gs_scholars_matched_with_twitter_accounts_500.csv")[:500]
gsid_2_tid_eval.pop(gsid_2_tid_eval.columns[0])
gsid_2_tid_eval['gs_id'] = gsid_2_tid_eval['url'].apply(lambda x : get_gs_id(x))
gsid_2_tid_eval['t_id'] =  gsid_2_tid_eval['url of their twitter'].apply(lambda x: get_t_id(x))
gsid_2_tid_eval.head()

,name,organization,url,url of their twitter,gs_id,t_id
0,Sebastian Gerke,Ree Technology,https://scholar.google.com/citations?hl=en&use...,https://twitter.com/sebgerke,DeP0anAAAAAJ,sebgerke
1,Yang Liu,"Computer Science, harbin institute of technology",https://scholar.google.com/citations?hl=en&use...,NaN,3IC_ZtkAAAAJ,NaN
2,Son N. Tran,University of Tasmania,https://scholar.google.com/citations?hl=en&use...,https://twitter.com/sondinhtran,h82eCK8AAAAJ,sondinhtran
3,Massimiliano Ciaramita,Google,https://scholar.google.com/citations?hl=en&use...,NaN,PwsKWfUAAAAJ,NaN
4,Marco Baity-Jesi,Eawag,https://scholar.google.com/citations?hl=en&use...,NaN,4S1Ajs8AAAAJ,NaN


# save_twitter_metainfo.py

In [ ]:
# current algorithm
# Step 1 -  search for GS ID name on twitter to get candidates. 
# Sort candidates on descending number of followers. 
# Write candidates in sorted order and GS ID in a file - ./intermediate_data.
import re
import tweepy
from twitter import *
import requests
import json
import csv
import jsonlines
import time
from tqdm import tqdm


def save_user_info(q, save_name, GS_ID, api):
    all_candidate = []
    # search the query
    users = api.search_users(q)

    for user in users:
        if user.followers_count > 10 and user.friends_count > 10:
            dic = user._json
            dic["GS_ID"] = GS_ID
            dic["GS_Name"] = q
            all_candidate.append(dic)

    newlist = sorted(all_candidate, key=lambda d: d['followers_count'], reverse=True)
    # print(newlist)
    with jsonlines.open(path + "intermediate_data/" + save_name + ".jsonl", 'a') as writer:
        # csv_writer = csv.writer(writer)
        for each in newlist:
            # print(each)
            # csv_writer.writerow(each)
            writer.write(each)

    with jsonlines.open(path + "intermediate_data/" + save_name + ".csv", 'a') as writer:
        writer.write(GS_ID)

In [ ]:
for i in range(10):
    save_user_info(gsid_2_tid_eval['name'][i], "search_info", gsid_2_tid_eval['gs_id'][i], api)
    # try:
    #     save_user_info(total_info[1][item], "search_info", total_info[0][item], api)
    # except:
    #     time.sleep(20)
    #     api, api2 = api2, api
    # time.sleep(2)

#sketch_match.py

In [ ]:
from sys import meta_path
import tweepy
import csv
import requests
import os
from deepface import DeepFace
from twitter import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from icecream import ic
import re
import jsonlines
import json

def download_profile_image(name, img_link):
    img_link = img_link.replace("normal", "400x400")
    if "default_profile_images" in img_link:
        return False
    if img_link:
        response = requests.get(img_link)
        file = open(path + "temp_pictures/" + name + '.jpg', "wb")
        file.write(response.content)
        return True
    return False

def compare_two_images(img1, img2):
    try:
        result = DeepFace.verify(img1, img2)
        if result["distance"] < 0.25:
            return True
        return False
    except:
        return False

def whether_same_image(img1, img2):
    # https://deepai.org/machine-learning-model/image-similarity
    r = requests.post(
        "https://api.deepai.org/api/image-similarity",
        files={
            'image1': open(img1, 'rb'),
            'image2': open(img2, 'rb'),
        },
        headers={'api-key': '8d00045d-7b5b-4b84-969f-f96d79a4a94c'}
    )
    if r.json()["output"]["distance"] == 0:
        return True
    return False

def match_by_meta_description(str1, str2):
    corpus = [str1, str2]
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')
    count_vectorizer = CountVectorizer(ngram_range=(1, 1), tokenizer=token.tokenize)
    vec = count_vectorizer.fit_transform(corpus)
    score = cosine_similarity(vec, vec)[0][1]
    if score > 0.399:
        return True
    return False

def get_keywords():
    field_names = ['natural language processing', 'computer vision', 'machine learning', 'artificial intelligence',
                   'deep learning', 'reinforcement learning', 'CV', 'NLP', 'AI', 'RL', 'GAN']
    role_names = ['researcher', 'research scientist', 'professor', 'Prof ', 'computational linguist']
    org_names = ['university']
    misc_names = ['#NLProc']
    additional_field_names = []
    for field in field_names:
        abbrev = ""
        for word in field.split():
            abbrev += word[0]

        additional_field_names += [field.replace(' ', '_'), field.replace(' ', ''), abbrev]
    field_names += additional_field_names
    keywords = field_names + role_names + org_names + misc_names
    return keywords


def if_keyword_match(text, keywords):
    text_lower = text.lower().split()
    return any(i in text_lower for i in keywords)


def match_by_homepage_link(str1, str2):
    corpus = [str1, str2]
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')
    count_vectorizer = CountVectorizer(ngram_range=(1, 1), tokenizer=token.tokenize, stop_words='english')
    vec = count_vectorizer.fit_transform(corpus)
    score = cosine_similarity(vec, vec)[0][1]
    if score > 0.9:
        return True
    return False


def match_by_tweets_arXiv(twitter_name):
    if os.path.exists(path + "/saved_user_tweets/" + twitter_name + ".csv"):
        with open(path + "/saved_user_tweets/" + twitter_name + ".csv", "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            for each_line in csv_reader:
                text = each_line[3]
                urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', text)
                for each in urls:
                    try:
                        r = requests.get(each)
                        text = text.replace(each, r.url)
                    except:
                        continue
                if "arXiv" in text:
                    return True
        return False
    return False


keywords = get_keywords()

def match_GS_with_Twitter(file_name, output_file, GS_image_folder):
    with open(output_file, 'w') as csv_write:
        # csv_reader = csv.reader(csv_file)
        import numpy as np
        meta_GS_info = np.load(file_name, allow_pickle=True)
        csv_writer = csv.writer(csv_write)
        for row in meta_GS_info:
            real_name = row['name']
            if "/" in real_name:
                continue
            # print(real_name)
            # search_result = twitter.users.search(q = real_name)
            with jsonlines.open(path + "saved_user_info/" + real_name + ".jsonl", 'r') as reader:
                found_match = False
                for each_candidate in reader:
                    candidate_screen_name = each_candidate["screen_name"]
                    Twitter_discription = each_candidate["description"]
                    if each_candidate["entities"]["description"]["urls"]:
                        description_link = each_candidate["entities"]["description"]["urls"][0]["display_url"]
                        Twitter_discription = re.sub(
                            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)',
                            description_link, Twitter_discription)
                    GS_meta_info = row['extra_info'][0]
                    similiar_description = match_by_meta_description(Twitter_discription, GS_meta_info)
                    if similiar_description:
                        csv_writer.writerow([real_name, candidate_screen_name, "match by meta description"])
                        found_match = True
                    if if_keyword_match(Twitter_discription, keywords):
                        csv_writer.writerow([real_name, candidate_screen_name, "match by keywords in meta description"])
                        found_match = True
                if not found_match:
                    csv_writer.writerow([real_name, 'Not Found'])


def match_single_author(GS_info, twiteer_meta_info):
    # return a Boolean, whether it can be matched
    candidate_screen_name = twiteer_meta_info["screen_name"]
    twitter_discription = twiteer_meta_info["description"]
    if twiteer_meta_info["entities"]["description"]["urls"]:
        description_link = twiteer_meta_info["entities"]["description"]["urls"][0]["display_url"]
        twitter_discription = re.sub(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)',
            description_link, twitter_discription)
    personal_webpage = GS_info.get("personal_webpage")
    try:
        candidate_homepage_link = twiteer_meta_info['entities']['url']['urls'][0]['expanded_url']
    except KeyError:
        candidate_homepage_link = None
    if personal_webpage and candidate_homepage_link:
        for i in personal_webpage:
            if match_by_homepage_link(candidate_homepage_link, i):
                return 1
    GS_meta_info = ""
    for i in GS_info['extra_info']:
        GS_meta_info+=i
    similiar_description = match_by_meta_description(twitter_discription, GS_meta_info)
    if if_keyword_match(twitter_discription, keywords):
        return 2
    if similiar_description:
        return 3
    return 0

# if __name__ == '__main__':
#     import numpy as np
#     meta_GS_info = np.load(path + "/data_from_GS/sample_info.npy", allow_pickle=True)
#     ans = 0
#     sum = 0
#     for row in meta_GS_info:
#         real_name = row['name']
#         with jsonlines.open("./saved_user_info/" + real_name + ".jsonl", 'r') as reader:
#             found_match = False
#             for each_candidate in reader:
#                 if match_single_author(row, each_candidate):
#                     ans+=1
#         sum += 1
#     print(ans)
#     print(sum)

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


#save_tweets.py 

In [ ]:
import re
import tweepy
from twitter import *
import requests
import json
import csv
import jsonlines
import time
from tqdm import tqdm
import time

def get_tweets_simple(twitter_name, api):
    try:
        tweets = api.user_timeline(screen_name=twitter_name, count=10, tweet_mode="extended")
        tweets_list = []
        for tweet in tweets:
            json_object = tweet._json
            tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
                          "text": json_object["full_text"]}
            # get full text from retweets
            if "retweeted_status" in json_object:
                tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
                              "text": json_object["retweeted_status"]["full_text"]}
            # change the form of link in the tweets
            urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', tweet_info["text"])
            for each in urls:
                try:
                    r = requests.get(each)
                    tweet_info["text"] = tweet_info["text"].replace(each, r.url)
                except:
                    continue
            tweets_list.append(tweet_info)
        return tweets_list
    except:
        return None


def get_tweets_simple_qnqq(twitter_name, api):
    try:
        tweets = api.user_timeline(screen_name=twitter_name, count=10, tweet_mode="extended")
        tweets_list = []
        for tweet in tweets:
            json_object = tweet._json
            tweets_list.append(json_object)
        return tweets_list
    except:
        return None

def get_tweets_qnqq(twitter_name, api):
    try:
        tweets = api.user_timeline(screen_name=twitter_name, count=100, tweet_mode="extended")
        tweets_list = []
        for tweet in tweets:
            json_object = tweet._json
            tweets_list.append(json_object)
        return tweets_list
    except:
        return None

def get_tweets(twitter_name, api):
    current_time = time.time()
    try:
        tweets = api.user_timeline(screen_name=twitter_name, count=300, tweet_mode="extended")
        print(f"time crawing one tweets is {time.time()-current_time}")
        current_time = time.time()
        tweets_list = []
        for tweet in tweets:
            json_object = tweet._json
            tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
                          "text": json_object["full_text"]}
            # get full text from retweets
            if "retweeted_status" in json_object:
                tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
                              "text": json_object["retweeted_status"]["full_text"]}
            # change the form of link in the tweets
            urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', tweet_info["text"])
            for each in urls:
                try:
                    r = requests.get(each)
                    tweet_info["text"] = tweet_info["text"].replace(each, r.url)
                except:
                    continue
            tweets_list.append(tweet_info)
        print(f"time processing one tweets is {time.time()-current_time}")
        return tweets_list
    except:
        return None

def get_full_tweets(twitter_name, api):
    pass

# if __name__ == "__main__":
#     import numpy as np
#     total_info = np.load("./data_from_GS/base_info_clean.npy", allow_pickle=True)
#     searched_dict = {}
#     for i in tqdm(range(100)):
#         get_tweets("aarnilmari", api)
#         time.sleep(1)

#match_and_save.py

In [ ]:
# Step 2 - Use match_and_save.py to make a sketch match by the type 1, 2, 3 match and save those users tweets
# type 1: matched by personal website
# type 2: matched by keyword
# type3: matched by similar description with the information in GS

import re
import tweepy
from twitter import *
import requests
import json
import csv
import jsonlines
import time
from tqdm import tqdm
import numpy as np
from icecream import ic

GS_info_path = path + "/gs_scholars_new.npy"
supplement_GS_info_path = path + "/data_from_GS/GS_info_beta.npy"
search_info_path = path + "intermediate_data/search_info.jsonl"
tweets_save_path = path + "final_output/tweets_data_website_matching.jsonl"
GS_save_path = path + "/final_output/GS_data_website_matching.jsonl"

tweets_dict = {}
with jsonlines.open(tweets_save_path, 'r') as reader:
    for i in reader:
        tweets_dict[i["screen_name"]] = 1
print("Finish load screen_name")

GS_info = np.load(GS_info_path, allow_pickle=True)
GS_info_dict = {}
for i in range(len(GS_info[0])):
    id = GS_info[1][i]
    GS_info_dict[id] = {
        "name":GS_info[0][i],
        "id":GS_info[1][i],
        "personal_webpage":GS_info[2][i],
    }
supplement_GS_info = np.load(supplement_GS_info_path, allow_pickle=True)
for i in supplement_GS_info:
    id = i.get('url').split("user=")[-1]
    if (GS_info_dict.get(id)==None):
        GS_info_dict[id] = i

current_time = 0
import time
last_time = time.time()
with jsonlines.open(search_info_path, 'r') as reader:
    now_id = ""
    candidate_list = []
    for each_candidate in reader:
        if each_candidate["GS_ID"] != now_id:
            GS_author_dict = {}
            GS_author_dict["id"] = now_id
            GS_author_dict["screen_name_list"] = []
            GS_author_dict["matched_type"] = []

            if (len(candidate_list)!=0):
                matched_list = []
                if (GS_info_dict.get(now_id)!=None):
                    GS_info_now = GS_info_dict[now_id]
                    for i in candidate_list:
                        if match_single_author(GS_info_now, i)>0:
                            matched_list.append([i, match_single_author(GS_info_now, i)])
                if (len(matched_list) == 0):
                    GS_author_dict["is_matched"] = False
                    for i in candidate_list:
                        screeen_name = i["screen_name"]
                        if (tweets_dict.get(screeen_name) != None):
                            continue
                        GS_author_dict["screen_name_list"].append(screeen_name)
                        GS_author_dict["matched_type"].append(0)
                        tweets_list = get_tweets_simple_qnqq(screeen_name, api)
                        if (tweets_list == None):
                            api = apis.change_api()
                        else:
                            current_time += 1
                            if current_time % 100 == 0:
                                print(f"time usage: {time.time() - last_time}")
                                last_time = time.time()
                            with jsonlines.open(tweets_save_path, 'a') as writer:
                                writer.write({"screen_name":screeen_name,"tweets":tweets_list})
                            del tweets_list
                            tweets_dict[screeen_name] = 1
                else:
                    GS_author_dict["is_matched"] = True
                    for i in matched_list:
                        screeen_name = i[0]["screen_name"]
                        if (tweets_dict.get(screeen_name) != None):
                            continue
                        GS_author_dict["screen_name_list"].append(screeen_name)
                        GS_author_dict["matched_type"].append(i[1])
                        tweets_list = get_tweets_qnqq(screeen_name, api)
                        if (tweets_list == None):
                            api = apis.change_api()
                        else:
                            current_time += 1
                            if current_time % 100 == 0:
                                print(f"time usage: {time.time() - last_time}")
                                last_time = time.time()
                            with jsonlines.open(tweets_save_path, 'a') as writer:
                                writer.write({"screen_name":screeen_name,"tweets":tweets_list})
                            del tweets_list
                            tweets_dict[screeen_name] = 1

            with jsonlines.open(GS_save_path, 'a') as writer:
                writer.write(GS_author_dict)

            del GS_author_dict
            del candidate_list
            candidate_list = [each_candidate]
            now_id = each_candidate["GS_ID"]
        else:
            candidate_list.append(each_candidate)